In [1]:
%load_ext autoreload
%autoreload 2

In [72]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import time as time
import re
from random import randint
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import logging
import time

In [3]:
import os

In [4]:
import sys
sys.path.append("../../code")
#sys.path.append("../../loaders")
import loaders.cityprotect as cp

Concatenate and pandaize data downloaded from CityProtect.
Given a folder, find all the files ending with "Los_Altos_Police_Department_report.csv", concatenate, and sort by date.

In [5]:
def geocode(geolocator, addr, sleep_sec):
    try:
        return geolocator.geocode(addr)
    except GeocoderTimedOut:
        logging.info('TIMED OUT: GeocoderTimedOut: Retrying...')
        time.sleep(randint(1*100,sleep_sec*100)/100)
        return geolocator.reverse_geocode(geolocator, latlon, sleep_sec)
    except GeocoderServiceError as e:
        logging.info('CONNECTION REFUSED: GeocoderServiceError encountered.')
        logging.error(e)
        return None
    except Exception as e:
        logging.info('ERROR: Terminating due to exception {}'.format(e))
        return None

In [16]:
import certifi
import ssl
import geopy.geocoders
from geopy.geocoders import Nominatim
from geopy.geocoders import Photon
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
user_agent = 'user_me_{}'.format(randint(10000,99999))
geolocator = Photon(user_agent=user_agent, scheme='http')



In [17]:
location = geolocator.reverse("48.8588443, 2.2943506")

print(location.address)
print (location.raw)

Pilier Nord Caisse, Quai Jacques Chirac, 75007, Quai Jacques Chirac, Paris, Île-de-France, France
{'geometry': {'coordinates': [2.2943973500000054, 48.8587893], 'type': 'Point'}, 'type': 'Feature', 'properties': {'osm_id': 335101035, 'extent': [2.2943659, 48.8588097, 2.2944288, 48.8587689], 'country': 'France', 'city': 'Paris', 'countrycode': 'FR', 'postcode': '75007', 'locality': 'Quartier du Gros-Caillou', 'type': 'house', 'osm_type': 'W', 'osm_key': 'shop', 'street': 'Quai Jacques Chirac', 'district': 'Paris', 'osm_value': 'ticket', 'name': 'Pilier Nord Caisse', 'state': 'Île-de-France'}}


In [66]:
bob = geolocator.geocode("1728 Selig Ln, Los Altos, CA")

In [67]:
print(bob.raw["properties"]["city"])

Los Altos


In [68]:
bob.address

'Selig Lane, 94807, Los Altos, California, United States'

In [7]:
previous_df = pd.read_pickle("../../data/clean_data/SCCSheriff_2017_2022_inferredCity.pkl")

In [93]:
extra_df = pd.read_pickle("SCCSheriff.pkl")

In [95]:
previous_df = pd.concat([previous_df, extra_df])

In [96]:
previous_df["date"].max()

Timestamp('2023-07-01 06:48:01')

In [97]:
dept = "Santa Clara County Office of the Sheriff"
df = cp.cityprotect("../../data/raw_data/{}".format(dept.replace(" ", "_")),
                                 start_date = previous_df["date"].max(),
                                  end_date = datetime.now() )

Starting: 2023-07-01 06:48:01
Ending:   2023-10-14 12:59:43.546747
Loading from 27 files.
Data file ../../data/raw_data/Santa_Clara_County_Office_of_the_Sheriff\Apr_Jun_2017_Santa_Clara_County_Office_of_the_Sheriff_report.csv is empty.
Data file ../../data/raw_data/Santa_Clara_County_Office_of_the_Sheriff\Jan_Mar_2017_Santa_Clara_County_Office_of_the_Sheriff_report.csv is empty.


In [98]:
df["date"].min()

Timestamp('2023-07-01 06:48:01')

In [99]:
df["inferredCity"]="unknown"

In [100]:
row = df.iloc[125]

In [101]:
row

ccn                                                          S231820338
date                                                2023-07-02 05:17:50
updateDate                                          2023-07-02 16:42:42
city                                                 SANTA CLARA COUNTY
state                                                                CA
postalCode                                                            .
blocksizedAddress                              200 Block N CLAREMONT AV
incidentType                                           ANIMAL COMPLAINT
parentIncidentType                                                Other
narrative             Call Type: 1091   <br>Description: ANIMAL COMP...
inferredCity                                                    unknown
Name: 260184, dtype: object

In [102]:
place = re.sub("BLOCK", "", row["blocksizedAddress"].upper()) + ", SANTA CLARA COUNTY, CA"

In [105]:
place

'200  N CLAREMONT AV, SANTA CLARA COUNTY, CA'

In [103]:
location = geocode(geolocator, place, 1)

In [104]:
location

Location(Hayes Mansion, 200, Edenvale Avenue, 95136, Edenvale Avenue, San Jose, CA, United States, (37.262228300000004, -121.82000020171506, 0.0))

In [76]:
def attempt(geolocator, place, times=3):
    to_raise = None
    for _ in range(times):
        try:
            return geocode(geolocator, place, 1)
        except Exception as err:
            print("error; trying again ({})".format(place))
            time.sleep(5)
            to_raise = err
    raise to_raise

In [108]:
counter = 0
for index, row in df[df["inferredCity"] == "unknown"].iterrows():
    if row["inferredCity"] == "unknown":
        #print(row)
        try:
            place = re.sub("BLOCK", "", row["blocksizedAddress"].upper()) + ", SANTA CLARA COUNTY, CA"
        except:
            print("Failed to scan: {}".format(row["blocksizedAddress"]))
            continue
        
        location = attempt(geolocator, place, times=10)
            
        counter = counter + 1
        if type(location) == str:
            
            if location != None:
                display_name = [s.strip() for s in location.raw["display_name"].split(",")]                         
                try:
                    county_idx = display_name.index("Santa Clara County")
                except ValueError:
                    print("SCC not found: {}".format(display_name))
                    df.at[index, "inferredCity"] = "not_found"
                    continue
                df.at[index, "inferredCity"] = display_name[county_idx-1]   
                df.at[index, "postalCode"] = display_name[-2]                            
                print("\n{} {}: {}, {}".format(counter, location.raw["display_name"], df.at[index,"inferredCity"], df.at[index, "postalCode"]))
            else:
                df.at[index, "inferredCity"] = "no resolution"
                print("NR: {} ".format(counter), end="")

        elif type(location) == geopy.location.Location:
            if "city" in location.raw["properties"]:
                df.at[index, "inferredCity"] = location.raw["properties"]["city"]
            else:
                df.at[index, "inferredCity"] = "no resolution"
            if "postcode" in location.raw["properties"]:
                df.at[index, "postalCode"] = location.raw["properties"]["postcode"]
            else:
                df.at[index, "postalCode"] = "n/a"
 
            print("\n{}: {} {}- {}, {}".format(counter, df.at[index, "date"], location.address, df.at[index,"inferredCity"], df.at[index, "postalCode"]))
        if counter/200 == round(counter/200):
            print("Saving...")
            df.to_pickle("SCCSheriff.pkl")
    else:
        print(".", end="")


error; trying again (5900  MONTEREY FRONTAGE RD, SANTA CLARA COUNTY, CA)
error; trying again (200   ALDERCROFT HEIGHTS RD, SANTA CLARA COUNTY, CA)
error; trying again (12000  GREEN HILLS CT, SANTA CLARA COUNTY, CA)
error; trying again (7800  ORION LN, SANTA CLARA COUNTY, CA)
error; trying again (10900  N WOLFE RD, SANTA CLARA COUNTY, CA)
error; trying again (19700  GRAYSTONE LN, SANTA CLARA COUNTY, CA)
error; trying again (19700  GRAYSTONE LN, SANTA CLARA COUNTY, CA)
error; trying again (22600  QUEENS OAK CT, SANTA CLARA COUNTY, CA)
Saving...
Failed to scan: nan

279: 2023-09-11 01:50:56 Arroyo Grande Way, 95032, Los Gatos, California, United States- Los Gatos, 95032

280: 2023-09-11 01:51:27 Alamitos, 95118, San Jose, California, United States- San Jose, 95118

281: 2023-09-11 02:13:35 Alum Rock, South Capitol Avenue, 95127, South Capitol Avenue, San Jose, California, United States- San Jose, 95127

282: 2023-09-11 02:15:24 Shoreline Amphitheatre, 1, Amphitheatre Parkway, 94043, Amphi

In [110]:
df["date"].min()

Timestamp('2023-07-01 06:48:01')

In [87]:

len(df[df["inferredCity"] == "unknown"])

638

In [ ]:
df.to_pickle("SCCSheriff_2023_part2.pkl")

In [ ]:
df["inferredCity"].value_counts()

In [ ]:
df[df["inferredCity" == "unknown"]]

        try: 
            location = geolocator.geocode(place)
        except GeocoderTimedOut as e:
            print("Location error: {}".format(e))
            naptime = 3+randint(100)
            print("Sleeping {} seconds.".format(naptime))
            time.sleep(naptime)
            user_agent = 'user_me_{}'.format(randint(10000,99999))
            geolocator = Nominatim(user_agent=user_agent)
            location = geolocator.geocode(place)
